In [1]:
import pandas as pd
import time
import random
from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import math

In [2]:
df = pd.read_json("job_postings_prepped.json")

## LSH **Calculations**


In [3]:
import json

k = 100 #TODO: find out optimal value
vector_dimensions = len(df['Vector'][0])
non_zero_values = len(df['Vector'][0]) #TODO: find out optimal value

def generate_hash_function(k,vector_dimensions,non_zero_values):
    hash_functions_list = []

    for i in range(k):

        hash_functions = np.zeros(vector_dimensions)
        selected_indexes = random.sample(range(vector_dimensions),non_zero_values)
        
        for index in selected_indexes:
            random_vector = np.random.randn(1)
            hash_functions[index] = random_vector

        hash_functions_list.append(hash_functions)

    return hash_functions_list

random_hash_functions = generate_hash_function(k,vector_dimensions,non_zero_values)
random_hash_functions_df = pd.DataFrame({'Hash_Functions': random_hash_functions})

random_hash_functions_df.to_json('LSH Search Results/LSH Random Hash Functions.json', index=False)


C:\Users\ragne\AppData\Local\Temp\ipykernel_1784\3778698625.py:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  hash_functions[index] = random_vector


In [4]:
# Perform dot product and get binary vectors
def generate_binary_vectors(input_vector,random_hash_functions_df):
  binary_vector = []
  for i,row in random_hash_functions_df.iterrows():

    input_vector = np.asarray(input_vector, dtype=np.float64)
    hash_function = np.asarray(row['Hash_Functions'], dtype=np.float64)
    
    dot_product = np.dot(input_vector,hash_function)
    # result = 0 if dot_product <= 0 else 1
    result = np.where(dot_product <= 0, 0, 1)
    binary_vector.append(result)
  return binary_vector

# Make a new column called Binary Vectors for all the binary vectors
# random_hash_functions_df = pd.read_json('LSH Search Results/LSH Random Hash Functions.json')
# df['Binary Vectors'] = df['Vector'].apply(lambda x : generate_binary_vectors(x,random_hash_functions_df))

In [5]:
# Perform linear search on binary vector using hamming distance Dh
def hamming_distance(query_vector,database_sample):
  mismatched_bits = 0
  for i in range(len(query_vector)):
    if query_vector[i] != database_sample[i]:
      mismatched_bits += 1
  return mismatched_bits

##### Find Best K Value Time from 10 to 100

In [ ]:
#Calculate K Value Time Comparison
# k_time_dict = {}
# vector_dimensions = len(df['Vector'][0])
# non_zero_values = len(df['Vector'][0]) #TODO: find out optimal value

# for k in range(10,101,10):
#     start = time.time()

#     random_hash_functions = generate_hash_function(k,vector_dimensions,non_zero_values)
#     binary_vectors_list = df['Vector'].apply( lambda x : one_hashing_function(x,random_hash_functions))

#     end = time.time()
#     results_time = end - start
#     k_time_dict[k] = results_time
#     k_df = pd.DataFrame(list(k_time_dict.items()), columns=['K Value', 'Time'])
#     k_df.to_csv(f'LSH Search Results/K_Value_Time.csv', index=False)


## Calculate 10 different queries accuracies

In [9]:
# Calculate 10 different queries accuracies
import os

query_dict = {'Q1': "resilient investment banker", 
              'Q2': "2 years experience product manager", 
              'Q3': "10 years risk analyst problem solver", 
              'Q4': "tax analyst for big company", 
              'Q5': "software engineer for google or amazon", 
              'Q6': "video editor for advertisements with 5 year experience",
              'Q7': "full time senior head nurse position",
              'Q8': "after school math and science tutor",
              'Q9': "dietitian for professional atheletes",
              'Q10': "costume designer and makeup artist"}

query_df = pd.DataFrame.from_dict(query_dict, orient='index', columns=['Query Text'])

# Combine the data from the DataFrame and query_dict into a single list
# all_data = list(df["Data"].array) + list(query_dict.values())

vectorizer = CountVectorizer()
matrix = vectorizer.fit_transform(list(query_dict.values()))
matrix = matrix.toarray()

# Separate the vectors for the DataFrame and query_dict
# df_vectors = matrix[:len(df)]
# query_dict_vectors = matrix[len(df):]

# df["Vector"] = [row.tolist() for row in df_vectors]

# Variables for LSH #TODO: EDIT THIS FOR ACCURACY
k = 100
vector_dimensions = len(df['Vector'][0])
non_zero_values = len(df['Vector'][0])

# Generate Random Hash Functions
# random_hash_functions = generate_hash_function(k,vector_dimensions,non_zero_values)
# random_hash_functions.to_csv('LSH Search Results/LSH Random Hash Functions.csv')

random_hash_functions_df = pd.read_json('LSH Search Results/LSH Random Hash Functions.json')

# # Do LSH Calculations for database
df['Binary Vectors'] = df['Vector'].apply( lambda x : generate_binary_vectors(x,random_hash_functions_df))
df.to_csv('LSH Search Results/LSH Database with Binary Vectors.csv')

# Do LSH Calculations for query vectors
query_dict_binary_vectors = [generate_binary_vectors(vector, random_hash_functions_df) for vector in matrix]

query_df['Vectors'] = [row.tolist() for row in matrix]
query_df['Binary Vectors'] = query_dict_binary_vectors

query_df.to_csv('LSH Search Results/LSH Query Vectors with Binary Vectors.csv')

ValueError: shapes (45,) and (105248,) not aligned: 45 (dim 0) != 105248 (dim 0)

In [143]:
query_df = pd.read_csv('LSH Search Results/LSH Query Vectors with Binary Vectors.csv')
df = pd.read_csv('LSH Search Results/LSH Database with Binary Vectors.csv')

In [145]:
# Specify the folder path
folder_path = 'LSH Search Results'

# Define the Excel file path
excel_file_path = os.path.join(folder_path, 'LSH_Search_Results.xlsx')

with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:     
    
    for query_name,query_row in query_df.iterrows():

        mismatched_list = []

        for i, row in df.iterrows():
            mismatched_list.append(hamming_distance(query_row['Binary Vectors'],row['Binary Vectors'])) 
        
        df['Distance'] = mismatched_list

        # Get only data and distance from query_name
        query_result_df = df[['Data', 'Distance']].copy()

        # Get the top 50 results based on the 'Distance' column
        # query_result_top = query_result_df.nsmallest(50, 'Distance') 
    
        # Loop through each unique query and create separate sheets
        query_result_df.to_excel(writer, sheet_name=f'Results_{query_name}', index=False)